In [ ]:
#IMPORTATION DE MODULES
#from textblob import TextBlob
import json

In [15]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE
import re
from textblob import TextBlob
import string


#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, auteur, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.auteur = auteur
        self.contenu = contenu
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()
    
    def Extraction_Sentiments(self): #A Faire (utilisation textblob)
        """"""
        return sentiments
    
    def Extraction_Hashtags():  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
    def Extraction_Mentions(self.contenu):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+',self)

        return mentions
    

def Nettoyage_Tweet(self.contenu):
    """Fonction qui supprime les caractères spéciaux d'un tweet"""
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self)
    return enlever_speciaux



def Zone_Atterisssage (Fichier):
    """Stockage du tweet dans le fichier json zone d'atterissage"""
    
    ZA=[]
    
    with open (Fichier,'r',encoding='utf-8') as f:
        donnee= json.load(f)
    
    for infos_tweets in donnee:
        auteur = infos_tweets.get("author_id")
        contenu = infos_tweets.get("text")
        tweet_auteur = {
            "id": auteur,
            "tweet": Nettoyage_Tweet(contenu)
        }
        ZA.append(tweet_auteur)

    with open('ZA.json','w', encoding='utf-8') as f:
        json.dump(ZA, f, indent=4, ensure_ascii=False)

           
        
#tweet = Tweet (auteur, contenu) # Utilisation de la class Tweet
        

# Process chaque tweet:
                
with open ('ZA.json','r') as file:
    tweets = json.load(file)


for tweet in tweets:
    print(f"Auteur : {tweet.auteur}")
    print(f"Contenu : {tweet.contenu}")
    print(f"Hashtags : {tweet.hashtags}")
    print(f"Mentions : {tweet.mentions}")
    print(f"Sentiment : {tweet.sentiment}")

# --> il faut charger ces données dans une date frame : df = pd.DataFrame(tweets)

# yo




SyntaxError: invalid syntax (515494132.py, line 26)

In [13]:

def Nettoyage_Tweet(tweet):
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', tweet)
    return enlever_speciaux

Nettoyage_Tweet("Goumin des éléphants joueurs la même fatigue même 😫 #twitter225")

'Goumin des lphants joueurs la mme fatigue mme  #twitter225'